task 2.1: Make a DataModel class that reads the output of the DataAquisition class (from task1) in its __init__()  

task 2.2: Make a function in DataModel to make a linear regression. I suggest you try to do it on your own with only vanilla python and the class notes. If you are lost, you can find practical info here on how to use already made libraries: https://www.geeksforgeeks.org/machine-learning/regularization-in-machine-learning/Lenker til en ekstern side.  The issue here will be data structure: np.array vs list of list vs pandas DataFrames.  

task 2.3: Make a function that split the data you got from DataAquisition into train, validation and test. Do it  with vanilla python. You need to make sure you understand the data structure.  

task 2.4: Make a function that computes MSE (make your own, don't copy from my notes :P )  

task 2.5: Make a function to make NN. It would be essentially a wrapper of other libraries, I suggest to use Keras:  https://www.geeksforgeeks.org/machine-learning/how-to-create-models-in-keras/  . You should have acquired enough notions to handle this tool.  

task 2.6: Make a function that does K_MEAN and GMM (we will discuss them next week)  

Once these methods (recipes) are done, you can now make cakes! :) :  

task 2.7: Make a linear regression on all your data (statistic ).  

task 2.8: Make a linear regression on all your train data and test it on your validation.  

task 2.9: Compute the MSE on your validation data.   

task 2.10: Try for different distribution of initial data point, (a) Discuss how different functions can be used in the linear regression, and different NN architecture. (b) Discuss how you can use the validation data for the different cases. (c) Discuss the different outcome from the different models when using the full dataset to train and when you use a different ML approach. (d) Discuss the outcomes you get for K-means and GMM. (e) Discuss how you can integrate supervised and unsupervised methods for your case.  

In [ ]:
import sys
from data_aquisition import DataAquisition as aq

class DataModel:
    def __init__(self):
        sys.path.append('../MOD550-task1') 

        self.data = aq().data

    

**Task 1**  


In [ ]:
dataw = DataModel()


File found: ../title.basics.tsv
File found: ../data/title.ratings.tsv


,Title ID,Type,Title,originalTitle,isAdult,Release year,endYear,Runtime (minutes),Genres,Rating,Number of votes
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,\N,1.0,"Documentary,Short",5.7,2181
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,\N,5.0,"Animation,Short",5.5,301
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892.0,\N,5.0,"Animation,Comedy,Romance",6.4,2248
3,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,\N,1.0,Short,6.2,2990
4,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894.0,\N,1.0,Short,5.0,220
